# 本チャプターの目次
1. データマネジメントとは何者のなのか？
2. メタデータとは？
3. ビッグデータ基盤にメタデータが必要な理由
4. メタデータを保存するメタデータストアとは
5. ビッグデータ基盤におけるメタデータの提供形態
6. 3種類存在するメタデータ
7. 次のセクションへの導入

# データマネジメントとは何者なのか？

データマネジメントとは、

# メタデータとは？


# ビッグデータ基盤にメタデータが必要な理由

# メタデータを保存するメタデータストアとは？

# ビッグデータ基盤におけるメタデータの提供形態

# 3種類存在するメタデータ

# 次セクションへの導入
次のセクションよりメタデータを解説及び、ところどころ算出していきます。
そのための準備を行います。

ここから、注意！！！！  
Mysqlのデータベース(とテーブル)とSparkのデータベース(とテーブル)2つが出てきます。  
それぞれ区別して記載してありますが、この2つは別物です。  
Sparkのデータベース(とテーブル)のテーブル定義などはMysqlに格納されていますが、実データはロケーション
「/Users/saitouyuuki/Desktop/src/pyspark_dataprofiling_dataquality/dataset/parquet/」に格納され分離されています。

## データの読み込みとテーブルの作成をします※
1. jinko.csvを読み込んで表示する
2. jinko.csvの内容を保存するSparkテーブルを作成する
3. jinko.csvのデータを2で作成したSparkテーブルにInsertする

※以下の講座で行った最初の準備の簡略版です。  
「データサイエンスのための前処理入門PythonとSparkで学ぶビッグデータエンジニアリング(PySpark) 速習講座」  
https://www.udemy.com/course/python-spark-pyspark/?referralCode=E67BF8B61F65866794EB


In [ ]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init()

In [ ]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .enableHiveSupport() \
    .getOrCreate()


# spark.xxxxxと記載することで処理を分散させることが可能です。

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col

struct = StructType([
    StructField("code", StringType(), False),
    StructField("kenmei", StringType(), False),
    StructField("gengo", StringType(), False),
    StructField("wareki", StringType(), False),
    StructField("seireki", StringType(), False),
    StructField("chu", StringType(), False),
    StructField("sokei", StringType(), False),
    StructField("jinko_male", StringType(), False),
    StructField("jinko_female", StringType(), False)
])
df=spark.read.option("multiLine", "true").option("encoding", "SJIS") \
    .csv("/Users/saitouyuuki/Desktop/src/pyspark_dataprofiling_dataquality/dataset/jinko.csv", header=False, sep=',', inferSchema=False,schema=struct)

data_t=df.filter("code!='都道府県コード'")

data_t.show()


In [ ]:
# 定義はMysqlに格納されています（次のセクションで確認を行います）
spark.sql(""" 
create database if not exists data_management_cruch_course;

""")

In [ ]:
# テーブルの作成をします。
# 今回はCSVに一致した「jinko_table」Sparkテーブルを作成します
# 定義はMysqlに格納されています（次のセクションで確認を行います）
spark.sql(""" 
CREATE TABLE IF NOT EXISTS data_management_cruch_course.jinko_table (code String, gengo String,wareki String,seireki String,chu String,sokei String,jinko_male String,jinko_femail String)
PARTITIONED BY (kenmei String)
STORED AS PARQUET
LOCATION '/Users/saitouyuuki/Desktop/src/pyspark_dataprofiling_dataquality/dataset/parquet/';
""")


In [ ]:
# 作成したSparkテーブルの一覧を確認することができます
# 定義はMysqlに保存されているのでMysqlから内部的には引っ張り出してきています。
spark.sql("show tables in   data_management_cruch_course").show(truncate=False)

In [ ]:
# テーブルの定義を確認することができます。
spark.sql("show create table data_management_cruch_course.jinko_table").show(truncate=False)

In [ ]:
# テーブルにデータを登録します
# insertIntoというメソッドがありますが使いづらいのでテンポラリテーブルを作成してそこからselect insert します

data_t.createOrReplaceTempView("jinko_table_tmp")

#「データサイエンスのための前処理入門PythonとSparkで学ぶビッグデータエンジニアリング(PySpark) 速習講座」  
#https://www.udemy.com/course/python-spark-pyspark/?referralCode=E67BF8B61F65866794EB
#より、パーティションとリパーティションをつけることを忘れずに
spark.sql(""" 
Insert into table data_management_cruch_course.jinko_table partition(kenmei)
/** REPARTITION(1) */
Select 
code,gengo,wareki,seireki,chu,sokei,jinko_male,jinko_female,kenmei
from jinko_table_tmp
  """)

In [ ]:
# データがしっかり入っているか確認してみましょう
spark.sql("""
select kenmei,count(*) from data_management_cruch_course.jinko_table group by kenmei
""").show()

In [ ]:
spark.stop()
spark.sparkContext.stop()